In [5]:

pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=f325ca7226fed35b329730572c17cb83853bbc7d3704bbf60621bed9c192888d
  Stored in directory: c:\users\deeks\appdata\local\pip\cache\wheels\b6\0d\90\0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
Successfully built termcolor
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import keras as k
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [61]:
def load_images_labels(path_to_metadata_json):
    file = open(path_to_metadata_json) # Open meta-data
    data = json.load(file) # load meta-data
    file.close() # close file :)
    images , category = [], [] # make lists to append images and categories to
    for i in range(20000): # 20,000 is the amount of images we have to work with 
        images.append(np.array(k.preprocessing.image.load_img(
            'C:/Users/deeks/Downloads/2020-02-14_InfraredSolarModules/InfraredSolarModules/'+ data[str(i)]['image_filepath'], color_mode = "grayscale"))) # load the image and append to list
        category.append(data[str(i)]['anomaly_class']) # load its class
    return np.array(images), np.array(category) # return np arrays for ease of use later

In [62]:
def preprocess_images_labels(images, category):
    images = images.reshape(20000, 40, 24, 1) # reshape to show color channels
    images = images / 255 # scale data
    X_train, X_test, y_train, y_test = train_test_split(images, category, test_size=0.2)
    labelencoder = LabelEncoder() # Make a label encoder 
    y_train = labelencoder.fit_transform(y_train) # encode labels to numbers
    y_test = labelencoder.transform(y_test)
    y_train = tf.keras.utils.to_categorical(y_train) # encode nubers to catagorical labels
    y_test = tf.keras.utils.to_categorical(y_test)
    return X_train, X_test, y_train, y_test

In [63]:
def define_model():
    """
    This model is a little less accurate than the best one I found.
    But it also has onlya quarter the paramenters so its a lot smaller.
    """
    model = k.Sequential()
    model.add(k.layers.Conv2D(filters=25, kernel_size=(3,3), strides=(1, 1), padding="valid", input_shape=(40, 24, 1)))
    model.add(k.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding="same"))
    model.add(k.layers.Activation('relu'))
    model.add(k.layers.Conv2D(filters=25, kernel_size=(3,3), strides=(1, 1), padding="valid"))
    model.add(k.layers.Activation('relu'))
    model.add(k.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding="same"))
    model.add(k.layers.Flatten())
    model.add(k.layers.Dense(675))
    model.add(k.layers.Activation('relu'))
    model.add(k.layers.Dense(12))
    model.add(k.layers.Activation('softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
    model.summary()
    return model

In [64]:
images, categories = load_images_labels('C:/Users/deeks/Downloads/2020-02-14_InfraredSolarModules/InfraredSolarModules/module_metadata.json')
# prrocess and train test split
X_train, X_valid, y_train, y_valid = preprocess_images_labels(images, categories)

In [65]:
# define model
model = define_model()
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 38, 22, 25)        250       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 19, 11, 25)       0         
 g2D)                                                            
                                                                 
 activation_20 (Activation)  (None, 19, 11, 25)        0         
                                                                 
 conv2d_11 (Conv2D)          (None, 17, 9, 25)         5650      
                                                                 
 activation_21 (Activation)  (None, 17, 9, 25)         0         
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 9, 5, 25)         0         
 g2D)                                                 

In [66]:
# fit
history = model.fit(X_train, y_train, epochs=20, batch_size=20, verbose=1, shuffle=True)

Epoch 1/20
800/800 [==============================] - 17s 20ms/step - loss: 1.7690 - accuracy: 0.5002
Epoch 2/20
800/800 [==============================] - 13s 16ms/step - loss: 1.5495 - accuracy: 0.5373
Epoch 3/20
800/800 [==============================] - 15s 19ms/step - loss: 1.2628 - accuracy: 0.6203
Epoch 4/20
800/800 [==============================] - 17s 21ms/step - loss: 1.1216 - accuracy: 0.6553
Epoch 5/20
800/800 [==============================] - 16s 19ms/step - loss: 1.0227 - accuracy: 0.6800
Epoch 6/20
800/800 [==============================] - 17s 21ms/step - loss: 0.9518 - accuracy: 0.6996
Epoch 7/20
800/800 [==============================] - 16s 21ms/step - loss: 0.8798 - accuracy: 0.7204
Epoch 8/20
800/800 [==============================] - 16s 19ms/step - loss: 0.8246 - accuracy: 0.7394
Epoch 9/20
800/800 [==============================] - 16s 20ms/step - loss: 0.7782 - accuracy: 0.7503
Epoch 10/20
800/800 [==============================] - 17s 21ms/step - loss: 0.725

In [67]:
# evaluate
model.evaluate(X_valid, y_valid, verbose=1)

125/125 [==============================] - 2s 10ms/step - loss: 0.9272 - accuracy: 0.7623


[0.9271612167358398, 0.7622500061988831]

In [206]:
from PIL import Image

im=Image.open(r'C:\Users\deeks\Downloads\2020-02-14_InfraredSolarModules\InfraredSolarModules\images\0.jpg')
im.show()

In [77]:
import matplotlib.pyplot as plt

In [407]:
im=plt.imread('C:/Users/deeks/Downloads/2020-02-14_InfraredSolarModules/InfraredSolarModules/images/8351.jpg')
im1 = im.reshape(1,40, 24, 1) # reshape to show color channels
im1 = im1 / 255 # scale data
m=model.predict(im1)


In [460]:
import csv
list1=[]
#with open('try.csv','w')as csvfile:
for i in range(20000):
    im=plt.imread('C:/Users/deeks/Downloads/2020-02-14_InfraredSolarModules/InfraredSolarModules/images/'+str(i)+'.jpg')
    im1 = im.reshape(1,40, 24, 1) # reshape to show color channels
    im1 = im1 / 255 # scale data
    m=model.predict(im1)
    list1.append([i,np.argmax(m)])
    #csvwriter=csv.writer(csvfile)
    #csvwriter.writerow(['file name','class name'])
    #csvwriter.writerows([i,np.argmax(m)])

file = open('try.csv', 'a+', newline ='')
  
# writing the data into the file
with file:    
    write = csv.writer(file)
    write.writerows(list1)

0


0


In [411]:
np.argmax(m)

11

In [412]:
categories

array(['Offline-Module', 'Offline-Module', 'Offline-Module', ...,
       'No-Anomaly', 'No-Anomaly', 'No-Anomaly'], dtype='<U14')

In [348]:
import pandas as pd 
df = pd.read_json('C:/Users/deeks/Downloads/2020-02-14_InfraredSolarModules/InfraredSolarModules/module_metadata.json')
df.to_csv('C:/Users/deeks/Downloads/2020-02-14_InfraredSolarModules/classes_data.csv')